<a href="https://colab.research.google.com/github/zLolz-ZK/skin-cancer-detection-website/blob/main/Main_Coding_Details.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the Packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Installing Kaggle and downloading the dataset from the api token key stored in Google Drive

In [ ]:
!pip install -q kaggle
from google.colab import files

!mkdir ~/.kaggle
!cp 'drive/MyDrive/Cancer Model/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!mkdir training
!cd training
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

#Unzipping the dataset and Extracting the data 

In [ ]:
from zipfile import ZipFile
file_name='skin-cancer-mnist-ham10000.zip'

with ZipFile(file_name,'r') as zipx:
  zipx.extractall()
  print('finished!!!')

finished!!!


#Using Pandas to see the Dataset CSV information and metadata. 

In [ ]:
df_data = pd.read_csv('HAM10000_metadata.csv')

df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


#Using Pandas to see the Dataset CSV information and metadata. 

In [ ]:
df_data = pd.read_csv('HAM10000_metadata.csv')

df_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


#Searching For unique lessions to eliminate duplicate data

In [ ]:
# identify lession and with lesion_id
df = df_data.groupby('lesion_id').count()
# filter out lesion_id's that have only one image associated with it
df = df[df['image_id'] == 1]
df.reset_index(inplace=True)
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000001,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1


In [ ]:
# this will tell us how many images are associated with each lesion_id
df = df_data.groupby('lesion_id').count()
# now we filter out lesion_id's that have only one image associated with it
df = df[df['image_id'] == 1]
df.reset_index(inplace=True)
df.head()

In [ ]:
def identify_duplicates(x):
    
    unique_list = list(df['lesion_id'])
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'
    
# create a new colum that is a copy of the lesion_id column
df_data['duplicates'] = df_data['lesion_id']
# apply the function to this new column
df_data['duplicates'] = df_data['duplicates'].apply(identify_duplicates)

df_data.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,duplicates
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,has_duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,has_duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,has_duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,has_duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,has_duplicates


In [ ]:
df_data['duplicates'].value_counts()

no_duplicates     5514
has_duplicates    4501
Name: duplicates, dtype: int64

In [ ]:
# now we filter out images that don't have duplicates
df = df_data[df_data['duplicates'] == 'no_duplicates']

df.shape

(5514, 8)

In [ ]:
# now we create a test set using df because we are sure that none of these images
# have augmented duplicates in the train set

y = df['dx']

_, df_test = train_test_split(df, test_size=0.20, random_state=101, stratify=y)
y = df_test['dx']
df_test, df_val = train_test_split(df_test, test_size=0.50, random_state=101, stratify=y)

df_test.shape, _.shape,df_val.shape

#Creating Train, Test and Validation Dir

In [ ]:
base_dir = 'base_dir'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

test_dir = os.path.join(base_dir, 'test_dir')
os.mkdir(test_dir)

val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)

classes=['nv','mel','bkl', 'bcc', 'akiec', 'vasc', 'df']

for cl in classes:
    cl = os.path.join(train_dir, cl)
    os.mkdir(cl)
for cl in classes:
    cl = os.path.join(test_dir, cl)
    os.mkdir(cl)
for cl in classes:
    cl = os.path.join(val_dir, cl)
    os.mkdir(cl)    

# Identifying  the train, test and validation data

In [ ]:
#identify test rows val
def identify_test_rows(x):
    # create a list of all the lesion_id's in the test set
    test_list = list(df_test['image_id'])
    val_list = list(df_val['image_id'])
    if str(x) in test_list:
        return 'test'
    elif str(x) in val_list:
        return 'val'
    else:
        return 'train'

# create a new colum that is a copy of the image_id column
df_data['train_test_val'] = df_data['image_id']
# apply the function to this new column
df_data['train_test_val'] = df_data['train_test_val'].apply(identify_test_rows)
   
# filter out train rows
df_train = df_data[df_data['train_test_val'] == 'train']

print(f'{len(df_train)} + {len(df_test)} + {len(df_val)} = {len(df_val)+len(df_test)+len(df_train)}')

In [ ]:
df_train['dx'].value_counts()
df_test['dx'].value_counts()
# Set the image_id as the index in df_data
df_data.set_index('image_id', inplace=True)

# Transferring the data

In [ ]:
# Get a list of images in each of the two folders test
folder_1 = os.listdir('ham10000_images_part_1')
folder_2 = os.listdir('ham10000_images_part_2')

# Get a list of train and test images
train_list = list(df_train['image_id'])
test_list = list(df_test['image_id'])
val_list = list(df_val['image_id'])

for image in train_list:    
    fname = image + '.jpg'
    label = df_data.loc[image,'dx']
    if fname in folder_1:
        # source path to image
        src = os.path.join('ham10000_images_part_1', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # source path to image
        src = os.path.join('ham10000_images_part_2', fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

# Transfer the test images
for image in test_list:
    fname = image + '.jpg'
    label = df_data.loc[image,'dx']
  
    if fname in folder_1:
        # source path to image
        src = os.path.join('ham10000_images_part_1', fname)
        # destination path to image
        dst = os.path.join(test_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # source path to image
        src = os.path.join('ham10000_images_part_2', fname)
        # destination path to image
        dst = os.path.join(test_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

for image in val_list:
    fname = image + '.jpg'
    label = df_data.loc[image,'dx']
    
    if fname in folder_1:
        # source path to image
        src = os.path.join('ham10000_images_part_1', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # source path to image
        src = os.path.join('ham10000_images_part_2', fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [ ]:
def num_of_images(train = '', test = '',val=''):
  base = 'base_dir'
  test_dir = 'test_dir'
  train_dir = 'train_dir'
  val_dir = 'val_dir'
  print(train_dir if train == 'train' else '', test_dir if test == 'test' else '',val_dir if val == 'val' else '' )
  for cl in classes:
    
    print(
        len(os.listdir(os.path.join(base, train_dir, cl))) if train == 'train' else '', 
        len(os.listdir(os.path.join(base, test_dir, cl))) if test == 'test' else '',
        len(os.listdir(os.path.join(base, val_dir, cl))) if val == 'val' else '',
          )
num_of_images('train','test','val')

train_dir test_dir val_dir
5364 1341 671
890 223 112
879 220 110
411 103 52
262 65 32
114 28 14
92 23 11


#Setting up the global Parameters

In [ ]:
train_path = 'base_dir/train_dir'
test_path = 'base_dir/test_dir'
valid_path = 'base_dir/val_dir'
num_train_samples = len(df_train)
num_test_samples = len(df_test)
num_val_samples = len(df_val)
train_batch_size = 32
test_batch_size = 32
val_batch_size = 32
image_size = 299#224,128

In [ ]:
for i in classes:
  print(f'{i} train -> {len(os.listdir("base_dir/train_dir/"+f"{i}"))}')
  print(f'{i} test -> {len(os.listdir("base_dir/test_dir/"+f"{i}"))}')
  print(f'{i} val -> {len(os.listdir("base_dir/val_dir/"+f"{i}"))}')

nv train -> 5364
nv test -> 1341
nv val -> 671
mel train -> 890
mel test -> 223
mel val -> 112
bkl train -> 879
bkl test -> 220
bkl val -> 110
bcc train -> 411
bcc test -> 103
bcc val -> 52
akiec train -> 262
akiec test -> 65
akiec val -> 32
vasc train -> 114
vasc test -> 28
vasc val -> 14
df train -> 92
df test -> 23
df val -> 11


#Creating Training Examples from augmented dimages to increase amount of training example to reduce overfitting and improve accuracy

In [ ]:
for item in classes:
    #classes=['nv','mel','bkl', 'bcc', 'akiec', 'vasc', 'df']
    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)
 
    # Choose a class
    img_class = item
 
    # list all images in that directory
    img_list = os.listdir('base_dir/train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir e.g. class 'mel'
    for fname in img_list:
            # source path to image
            src = os.path.join('base_dir/train_dir/' + img_class, fname)
            # destination path to image
            dst = os.path.join(img_dir, fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)
 
 
    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = 'base_dir/train_dir/' + img_class
 
    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=(0.9,1.1),
        #fill_mode='bilinear',
        #shear_range=0.2,
        )
 
    batch_size = 50
 
    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size)
 
 
 
    # Generate the augmented images and add them to the training folders
    
    ###########
    
    num_aug_images_wanted = 2000 # total number of images we want to have in each class
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))
 
    # run the generator and create about 6000 augmented images
    for i in range(0,num_batches):
 
        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    shutil.rmtree('aug_dir')

Found 6042 images belonging to 1 classes.
Found 1079 images belonging to 1 classes.
Found 1033 images belonging to 1 classes.
Found 488 images belonging to 1 classes.
Found 304 images belonging to 1 classes.
Found 132 images belonging to 1 classes.
Found 109 images belonging to 1 classes.


# Setting up The Image Data Generator for augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
transformation_ratio = .05
datagen = ImageDataGenerator(
    #rescale=1./255,
    preprocessing_function = tf.keras.applications.inception_resnet_v2.preprocess_input,
    rotation_range=180,
    zoom_range=[0.8,1.2],
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=2,#-1,0,1 possible vals
    height_shift_range=2,#-1,0,1
    brightness_range=(0.9,1.1),
    shear_range=0.1,
    )

train_batches = datagen.flow_from_directory(
      train_path,
      target_size=(image_size,image_size),
      batch_size=train_batch_size,
      shuffle = True,
      class_mode = 'sparse'
      )
datagen2 = ImageDataGenerator(
    tf.keras.applications.inception_resnet_v2.preprocess_input,
)
valid_batches = datagen2.flow_from_directory(
    valid_path,
    target_size=(image_size,image_size),
    batch_size=val_batch_size,
    shuffle = True,
    class_mode = 'sparse'
    )

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen2.flow_from_directory(
    valid_path,
    target_size = (image_size,image_size),
    batch_size = 1,
    shuffle = False,
    class_mode = 'sparse'
    )
num_of_images(train='train',test='test',val='val')

In [ ]:
def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None): # 12,6
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
#plots(imgs, titles=None) # titles=labels will display the image labels

In [ ]:
#Creating the Model
#iresnetv2 = iresnetv2.layers[-2].output
iresnetv2 = tf.keras.applications.InceptionResNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(image_size, image_size, 3),
    include_top=False,
)

In [ ]:
#iresnetv2.layers.pop()
iresnetv2.summary()

In [ ]:
for i in iresnetv2.layers[:400]:
  i.trainable = False
for i in iresnetv2.layers[400:]:
  #if isinstance(i,tf.keras.layers.BatchNormalization):
  i.trainable = True

In [ ]:
#model 1 acc 70% val accu 81% loss 0.8~0.9
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import  Model
model2 = tf.keras.models.Sequential([
                                iresnetv2,
                                GlobalAveragePooling2D(),
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(64,activation='relu'),
                                tf.keras.layers.Dropout(0.5),
                                tf.keras.layers.Dense(7,activation='softmax')
])


#model = Model(vgg.input, predictions)

In [ ]:
model2.compile(optimizer='nadam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['acc']
              )

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

# Setting up the model parameters and automated supervision

In [ ]:
file_path = os.getcwd()

def scheduler(epoch, lr):
   if epoch%3!=0:
     return lr
   else:
     return lr * 0.9

top_weights_path = os.path.join(os.path.abspath(file_path), 'dense_03_06.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(top_weights_path, monitor='val_acc',
                                        verbose=1, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=0),
    tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0),
    
]


In [ ]:
epochs  = 40
history = model.fit(
    train_batches, 
    steps_per_epoch=train_steps,
    validation_data=valid_batches,
    validation_steps=val_steps,
    epochs=epochs,
     callbacks = callbacks_list 
)

In [ ]:
loss, acc = \
model.evaluate(test_batches)

# Fine Training The model to imporove its accuracy by training the extractior

In [ ]:
l = 100

for layer in dense_model.layers[l:]:
  if not isinstance(layer, layers.BatchNormalization):
    layer.trainable = True
top_weights_path = os.path.join(os.path.abspath(file_path), 'dense_model_fine.h5')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(top_weights_path, monitor='val_acc',
                                        verbose=1, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=0),
    tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0),
]

In [ ]:
base_learning_rate=0.0005
model.compile(optimizer=tf.keras.optimizers.Nadam(base_learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['acc']
              )

In [ ]:
epochs  = 100
history_fine = model.fit(
    train_batches, 
    initial_epoch=history.epoch[-1],
    steps_per_epoch=train_steps,
    validation_data=valid_batches,
    validation_steps=val_steps,
    epochs=epochs,
     callbacks = callbacks_list 
)

In [ ]:
loss, acc = \
model.evaluate(test_batches)

#THis is Part 2 Testing Saved Model. 

In [ ]:
iresv2 = tf.keras.models.load_model('drive/MyDrive/Cancer Model/iresnetv2_fine.h5').get_layer('inception_resnet_v2')
cmodel = tf.keras.models.load_model('drive/MyDrive/Cancer Model/iresnetv2_fine.h5', compile=True)

In [ ]:
model_builder = keras.applications.inception_resnet_v2.InceptionResNetV2
img_size = (299, 299)
inputs = tf.keras.Input(shape=(299, 299, 3))
model = model_builder(weights="imagenet", include_top=True, input_tensor=inputs)
for layer, dense_layer in zip(model.layers[1:], iresv2.layers[1:]):
    layer.set_weights(dense_layer.get_weights())


225214464/225209952 [==============================] - 2s 0us/step


In [ ]:
img_size = (299, 299)
preprocess_input = keras.applications.inception_resnet_v2.preprocess_input

last_conv_layer_name = "conv_7b"
classes=['nv','mel','bkl', 'bcc', 'akiec', 'vasc', 'df']

In [ ]:
image_size = 299
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.inception_resnet_v2.preprocess_input)
test_batches = datagen.flow_from_directory(
    'drive/MyDrive/Cancer Model/val_dir',
    target_size = (image_size,image_size),
    batch_size = 1,
    shuffle = False,
    class_mode = 'sparse'
    )
cmodel.evaluate(test_batches,steps=828)

Found 828 images belonging to 7 classes.
828/828 [==============================] - 449s 538ms/step - loss: 0.2703 - acc: 0.9239


[0.27027857303619385, 0.9239130616188049]

#Part 3 visualization

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#The Grad-CAM algorithm

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

In [ ]:
cmodel = tf.keras.models.load_model('drive/MyDrive/Cancer Model/iresnetv2_fine.h5', compile=True)

In [ ]:
iresv2 = tf.keras.models.load_model('drive/MyDrive/Cancer Model/iresnetv2_fine.h5').get_layer('inception_resnet_v2')

In [ ]:
model_builder = keras.applications.inception_resnet_v2.InceptionResNetV2
img_size = (299, 299)
inputs = tf.keras.Input(shape=(299, 299, 3))
model = model_builder(weights="imagenet", include_top=True, input_tensor=inputs)
for layer, dense_layer in zip(model.layers[1:], iresv2.layers[1:]):
    layer.set_weights(dense_layer.get_weights())


225214464/225209952 [==============================] - 2s 0us/step


In [ ]:
model.summary()

In [ ]:
relu = model.get_layer(model.layers[-3].name)

x = tf.keras.layers.GlobalAveragePooling2D()(relu.output)

outputs = tf.keras.layers.Dense(7)(x)
model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

In [ ]:
ires = cmodel.layers[0]
ires.layers[-3].name

'conv_7b'

In [ ]:
img_size = (299, 299)
preprocess_input = keras.applications.inception_resnet_v2.preprocess_input

last_conv_layer_name = "conv_7b"
classes=['nv','mel','bkl', 'bcc', 'akiec', 'vasc', 'df']

In [ ]:
image_size = 299
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

# The local path to our target image
img_path = "drive/MyDrive/Cancer Model/val_dir/vasc/ISIC_0026490.jpg"
image = tf.keras.preprocessing.image.load_img(img_path)
input_arr = keras.preprocessing.image.img_to_array(image, dtype='float32')
input_arr = preprocess_input(input_arr)
input_arr = tf.keras.preprocessing.image.smart_resize(input_arr, size=(299,299))

#print(input_arr.shape)
n = np.argmax(cmodel.predict(np.array([input_arr])))
print(classes[n])
Image(img_path)

In [ ]:
cmodel.layers[0].inputs

[<KerasTensor: shape=(None, 299, 299, 3) dtype=float32 (created by layer 'input_4')>]

In [ ]:
# Prepare image
#img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Print what the top predicted class is

#preds = np.argmax(cmodel.predict(np.array([input_arr])))
print("Predicted:", classes[n])
model.layers[-1].activation = None
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
save_and_display_gradcam(img_path, heatmap)

# New Section